In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, Flatten, Conv2D, MaxPooling2D

Using TensorFlow backend.


In [2]:
train_path='Mask_Datasets\Train'
test_path='Mask_Datasets\Validation'


train_batches=ImageDataGenerator(rescale=1./255).flow_from_directory(
        train_path,
        color_mode='grayscale',
        target_size=(150, 150),
        shuffle=True,
        classes=['Mask','No_mask'])
            
test_batches=ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_path,
        target_size=(150, 150),
        shuffle=True,
        color_mode='grayscale',
        classes=['Mask','No_mask'])
epochs=30
width,height=150,150
num_features=64

Found 750 images belonging to 2 classes.
Found 350 images belonging to 2 classes.


In [2]:



model=Sequential()

model.add(Conv2D(num_features,(3,3),activation='relu',input_shape=(width,height,1)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(num_features,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(2*num_features,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(2*num_features,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(120,activation='relu'))
model.add(Dense(2,activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']
             )
             
history=model.fit_generator(train_batches,epochs=25,
                           validation_data=test_batches,verbose=1,shuffle=True)       

from keras.models import load_model

model.save('mask.h5') 


Found 750 images belonging to 2 classes.
Found 350 images belonging to 2 classes.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 64)      640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 17, 17, 128)       0         
______________________________________

In [10]:
import os
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

#load model
model = load_model('mask.h5')

face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


cap=cv2.VideoCapture(0)

while True:
    ret,test_img=cap.read()
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img,1.32,5)


    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        roi_gray=gray_img[y:y+w,x:x+h]
        roi_gray=cv2.resize(roi_gray,(150,150))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255
        predictions = model.predict(img_pixels)

        #find max indexed array
        max_index = np.argmax(predictions[0])

        emotions = ('Mask','No Mask')
        predicted_emotion = emotions[max_index]

        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('Mask or no mask detection',resized_img)
    if cv2.waitKey(1) & 0xFF==ord('q'):
        break

cap.release()
cv2.destroyAllWindows()